코드 4-1 파이토치를 사용한 MLP

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim) 입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스 엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim) 입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)

        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output


코드 4-2 MLP 객체 생성

In [ ]:
batch_size = 2 # 한 번에 입력할 샘플 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

코드4-3 랜덤한 입력으로 MLP 테스트하기

In [ ]:
import torch

def describe(x):
    print("타입: {}".format(x.type))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))

x_input = torch.rand(batch_size, input_dim)
describe(x_input)

In [ ]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

In [ ]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

### 4.2 예제: MLP로 성씨 분류하기

### 4.2.1 성씨 데이터셋

#### 코드4-5 SurnameDataset.__getitem__()구현

In [ ]:
class SurnameDataset(Dataset):
    # [코드 3-14]와 구현이 매우 비슷합니다.

    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        surname_vector = \
            self._vectorizer.vectorize(row.surname)
        nationality_index = \
            self._vectorizer.nationality_vocab.lookup_token(row.nationality)

        return {'x_surname': surname_vector,
                'y_nationality': nationality_index}


#### 코드 4-6 SurnameVectorizer 구현

In [ ]:
class SurnameVectorizer(object):
    """ 어휘 사전을 생성하고 관리합니다 """
    def __init__(self, surname_vocab, nationality_vocab):
        self.surname_vocab = surname_vocab
        self.nationality_vocab = nationality_vocab

    def vectorize(self, surname):
        """ 성씨에 대한 원-핫 벡터를 만듭니다

        매개변수:
            surname (str): 성씨
        반환값:
            one_hot (np.ndarray): 원-핫 벡터
        """
        vocab = self.surname_vocab
        one_hot = np.zeros(len(vocab), dtype=np.float32)
        for token in surname:
            one_hot[vocab.lookup_token(token)] = 1
        return one_hot


        @classmethod
        def from_dataframe(cls, surname_df):
            """ 데이터셋 데이터프레임에서 Vectorizer 객체를 만듭니다

            매개변수:
                surname_df (pandas.DataFrame): 성씨 데이터셋
            반환값:
                SurnameVectorizer 객체
            """
            surname_vocab = Vocabulary(unk_token="@")
            nationality_vocab = Vocabulary(add_unk=False)

            for index, row in surname_df.iterrows():
                for letter in row.surname:
                    surname_vocab.add_token(letter)
                nationality_vocab.add_token(row.nationality)

            return cls(surname_vocab, nationality_vocab)

#### 코드 4-7 MLP 기반의 SurnameClassifier

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SurnameClassifier(nn.Module):
    """ 성씨 분류를 의한 MLP """
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫번째 Linear 층의 출력 크기
            output_dim (int): 두번째 Linear 층의 출력 크기
        """
        super(SurnameClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """ MLP의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim) 입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스 엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim) 입니다.
        """
        intermediate_vector = F.relu(self.fc1(x_in))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector


#### 코드 4-8. MLP 기반의 성씨 분류기를 위한 하이퍼파라미터 프로그램 설정

In [ ]:
args = Namespace(
    # 날짜와 경로 정보
    surname_csv = "data/surnames/surnames_with_splits.csv",
    vectorizer_file = "vectorizer.json"
    model_state_file="model.pth",
    save_dir = "model_storage/ch4/surname_mlp",
    # 모델 하이퍼파라미터
    hidden_dim = 300,
    # 훈련 하이퍼파라미터
    seed = 1337,
    num_epochs = 100,
    early_stopping_criteria=5,
    learning_rate = 0.001
    batch_size = 64,
    # 실행 옵션은 주피터 노트북을 참고하세요
)

#### 코드 4-9. 데이터셋, 모델, 손실, 옵티마이저 생성

In [ ]:
dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
vectorizer = dataset.get_vectorizer()

classifier = SurnameClassifier(input_dim=len(vectorizer.surname_vocab),
                               hidden_dim=args.hidden_dim,
                               output_dim=len(vectorizer.nationality_vocab))

classifier = classifier.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)


#### 코드 4-10. 훈련 반복 코드의 일부

In [ ]:
# 훈련 과정은 5단계입니다

# --------------------------------------------
# 1단계. 그레이디언트를 0으로 초기화합니다
optimizer.zero_grad()

# 2단계. 출력을 계산합니다
y_pred = classifier(batch_dict['x_surname'])

# 3단계. 손실을 계산합니다
loss = loss_func(y_pred, batch_dict['y_nationality'])
loss_batch = loss.to("cpu").item()
running_loss += (loss_batch - running_loss) / (batch_index + 1)

# 4단계. 손실을 사용해 그레이디언트를 계산합니다
loss.backward()

# 5단계. 옵티마이저로 가중치를 업데이트합니다
optimizer.step()


#### 코드 4-11. 기존 모델(분류기)을 사용한 추론: 주어진 이름의 국적 예측하기


In [ ]:
def predict_nationality(name, clasifier, vectorizer):
    vectorized_name = vectorizer.vectorize(name)
    vectorized_name = torch.tensor(vectorized_name).view(1, -1)
    result = classifier(vectorized_name, apply_softmax=True)

    probability_values, indices = result.max(dim=1)
    index = indices.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)
    probability_value = probability_values.item()

    return {'nationality': predicted_nationality,
            'probability': probability_value}

#### 코드 4-12. 새로운 성씨에 대해 최상위 k개 예측 만들기

In [ ]:
def predict_topk_nationality(name, classifier, vectorizer, k=5):
    vectorized_name = vectorizer.vectorize(name)
    vectorized_name = torch.tensor(vectorized_name).view(1, -1)
    prediction_vector = classifier(vectorized_name, apply_softmax=True)
    probability_values, indices = torch.topk(prediction_vector, k=k)

    # 반환되는 크기는 (1, k) 입니다
    probability_values = probability_values.detach().numpy()[0]
    indices = indices.detach().numpy()[0]

    results = []
    for prob_value, index in zip(probability_values, indices):
        nationality = vectorizer.nationality_vocab.lookup_index(index)
        results.append({'nationality': nationality,
                        'probability': prob_value})

        return results

### 코드 4-13. 드롭아웃을 적용한 MLP

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """ MLP의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim) 입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스 엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim) 입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate, p=0.5))

        if apply_softmax:
            output = F.softmax(output, dim = 1)
        return output


### 코드 4-14 인공 데이터와 Conv1d 클래스

In [ ]:
import torch

batch_size = 2
one_hot_size = 10
sequence_width = 7
data = torch.randn(batch_size, one_hot_size, sequence_width)
conv1 = Conv1d(in_channels=one_hot_size, out_channels = 16,
               kernel_size = 3)
intermediate1 = conv1(data)
print(data.size())
print(intermediate1.size())

### 코드 4-15 데이터에 반복 적용한 합성곱

In [ ]:
import torch.nn as nn

conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

intermediate2 = conv2(intermediate1)
intermediate3 = conv3(intermediate2)

print(intermediate2.size())
print(intermediate3.size())



### 코드 4-16 특성 벡터를 줄이는 두가지 추가 방법

In [ ]:
# 특성 벡터를 줄이는 방법 2
print(intermediate1.view(batch_size, -1).size())

# 특성 벡터를 줄이는 방법 32
print(torch.mean(intermediate1, dim=2).size())
# print(torch.max(intermediate1, dim=2).size())
# print(torch.sum(intermediate1, dim=2).size())



## 4.4 예제: CNN으로 성씨 분류하기

### 4.4.1 SurnameDataset

### 코드 4-17 최대 성씨 길이를 전달하는 SurnameDataset

In [ ]:
class SurnameDataset(Dataset):
    # 4.2절 '"예제: MLP로 성씨 분류하기(4.2절)" 절의 구현 ...

    def __getitem__(self, index):
        row = self._target_df.iloc[index]

        surname_matrix = \
            self._vectorizer.vectorize(row.surname, self._max_seq_length)

        nationality_index = \
            self._vectorizer.nationality_vocab.lookup_token(row.nationality)

        return {'x_surname': surname_matrix,
                'y_nationality': nationality_index}


### 4.4.2 Vocabulary, Vectorizer, DataLoader
### 코드 4-18 CNN을 위한 SurnameVectorizer 구현

In [ ]:
clss SurnameVectorizer(object):
    """ 어휘 사전을 생성하고 관리합니다 """
    def vectorize(self, surname):
        """ 성씨에 대한 원-핫 벡터를 만듭니다

        매개변수:
            surname (str): 성씨
        반환값:
            one_hot (np.ndarray): 원-핫 벡터의 정렬
        """
        one_hot_matrix_size = (len(self.character_vocab), self.max_surname_length)
        one_hot_matrix = np.zeros(one_hot_matrix_size, dtype=np.float32)
        for position_index, character in enumerate(surname):
            character_index = self.character_vocab.lookup_token(character)
            one_hot_matrix[character_index][position_index] = 1

        return one_hot_matrix

    @classmethod
    def from_dataframe(cls, surname_df):
        """ 데이터셋 데이터프레임에서 Vectorizer 객체를 만듭니다

        매개변수:
            surname_df (pandas.DataFrame): 성씨 데이터셋
        반환값:
            SurnameVectorizer 객체
        """
        character_vocab = Vocabulary(unk_token="@")
        nationality_vocab = Vocabulary(add_unk=False)
        max_surname_length = 0

        for index, row in surname_df.iterrows():
            max_surname_length = max(max_surname_length, len(row.surname))
            for letter in row.surname:
                character_vocab.add_token(letter)
            nationality_vocab.add_token(row.nationality)

        return cls(character_vocab, nationality_vocab, max_surname_length)


### 코드 4-19 CNN 기반의 SurnameClassifier

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SurnameClassifier(nn.Module):
    def _init_(self, initial_num_channels, num_classes, num_channels):
        """
        매개변수:
            initial_num_channels (int): 입력 특성 벡터의 크기
            num_classes (int): 출력 예측 벡터의 크기
        """
        super(SurnameClassifier, self).__init__()

        self.convnet = nn.Sequential(
            nn.Conv1d(in_channels=initial_num_channels,
                      out_channels=num_channels, kernel_size=3),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels,
                      kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels,
                      kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels,
                      kernel_size=3),
            nn.ELU()
        )
        self.fc = nn.Linear(num_channels, num_classes)

    def forward(self, x_surname, apply_softmax=False):
        """ 모델의 정방향 계산

        매개변수:
            x_surname (torch.Tensor): 입력 데이터 텐서
                x_surname.shape은 (batch, initial_num_channels, max_surname_length) 입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스 엔트로피 손실을 사용하려면 False로 지정해야 합니다.
            반환값:
                결과 텐서. tensor.shape은 (batch, num_classes)입니다.
        """
        features = self.convnet(x_surname).squeeze(dim2)
        prediction_vector = self.fc(features)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector


### 코드 4-20 CNN 성씨 분류기의 입력 매개변수

In [ ]:
args = Namespace(
    # 날짜와 경로 정보
    surname_csv="data/surnames/surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch4/cnn",
    # 모델 하이퍼파라미터
    hidden_dim=100,
    num_channels=256,
    # 훈련 하이퍼파라미터
    seed=1337,
    learning_rate=0.001,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=5,
    dropout_p=0.1,
    # 실행 옵션은 주피터 노트북을 참고하세요.
)